In [1]:
import pandas as pd
import os
import csv
import xlsxwriter

In [2]:
#Primer paso: unificar los archivos csv en una lista de listas
ruta_archivos = r"C:\Users\Usuario\Desktop\Programación\Proyecto Análisis de Datos Entrenos\Datos entrenos nuevos\Sentadilla Frontal"
nombre_archivo = "Datos Sentadilla Frontal.xlsx"

def unificar_archivos(ruta_archivos):
    lista_datos = []
    for archivo in os.listdir(ruta_archivos):
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(ruta_archivos, archivo)
            with open(ruta_archivo, newline='') as archivo_csv:
                lector_csv = csv.reader(archivo_csv)
                for fila in lector_csv:
                    lista_datos.append(fila)
    return lista_datos

lista_datos = unificar_archivos(ruta_archivos)

In [3]:
#Segundo paso: convertir la lista de listas en un dataframe de pandas
def convertir_a_pandas(lista_datos):
    df = pd.DataFrame(lista_datos)
    return df

df = convertir_a_pandas(lista_datos)

In [4]:
#Paso 3: eliminar las filas que no nos interesan
df.drop(df.loc[df[0] == "Atleta"].index, inplace=True)
df.drop(df.loc[df[0] == "Marcado con *: Solo encoder con Bluetooth"].index, inplace=True)
df.drop(df.loc[df[0] == "Email"].index, inplace=True)

In [5]:
#Paso 4: Determinar que nombre quiero para mis columnas y elimnar esas filas
df.columns = df.iloc[0]
df.drop(df.loc[df["Ejercicio"] == "Ejercicio"].index, inplace=True)

In [6]:
#Paso 5: Eliminar las columnas que no me interesan
df.drop("Comentarios", axis=1, inplace=True)

In [7]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [8]:
#Paso 6: Igualar todas las filas en las que no haya datos en la columna ejercicio, fecha, etc.

def rellenar_datos_faltantes(columna):
    valor_del_momento = df.loc[0, columna]
    for indice, valor in df[columna].iteritems():
        if valor == "":
            df.loc[indice, columna] = valor_del_momento
        else:
            valor_del_momento = valor

for columna in df.columns:
    rellenar_datos_faltantes(columna)

In [9]:
#Cambio la fecha a formato date, elimino la hora
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d-%m-%Y %H:%M")
df["Fecha"] = df["Fecha"].dt.date


In [10]:
#Elimino las filas que no tienen datos en la columna serie y en la columna repeticion
df = df.dropna(subset=["nº Serie"])
df = df.dropna(subset=["nº Rep."])


In [11]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [12]:
#Transformo los datos de las columnas que van a contener datos numericos
columnas_no_numericas = ["Ejercicio", "Fecha", "Tipo*"]
for columna in df.columns:
    if columna not in columnas_no_numericas:
        df[columna] = df[columna].str.replace(",", ".")
        df[columna] = pd.to_numeric(df[columna])


In [13]:
#Convierto columna fecha a formato date
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y-%m-%d")

In [15]:
df

3,Ejercicio,Fecha,nº Serie,Peso (kg),nº Rep.,Tipo*,VMP (m/s),Velocidad máxima (m/s),Potencia Media (W),ROM (cm),1RM (kg)
0,SENTADILLA FRONTAL,2023-01-30,1,20.0,1,C,1.23,2.10,241.33,100.90,44.99
1,SENTADILLA FRONTAL,2023-01-30,2,50.0,1,C,0.86,1.51,421.83,86.51,73.95
2,SENTADILLA FRONTAL,2023-01-30,2,50.0,2,C,0.90,1.67,441.45,89.83,76.79
3,SENTADILLA FRONTAL,2023-01-30,2,50.0,3,C,0.89,1.62,436.54,92.30,76.06
4,SENTADILLA FRONTAL,2023-01-30,2,50.0,4,C,0.90,1.58,441.45,93.71,76.79
...,...,...,...,...,...,...,...,...,...,...,...
100,SENTADILLA FRONTAL,2023-03-06,2,85.0,2,C,0.41,1.02,341.88,81.04,88.74
101,SENTADILLA FRONTAL,2023-03-06,3,80.0,1,C,0.57,1.26,447.34,82.65,93.28
102,SENTADILLA FRONTAL,2023-03-06,3,80.0,2,C,0.50,1.08,392.40,83.30,88.74
103,SENTADILLA FRONTAL,2023-03-06,4,85.0,1,C,0.59,1.30,491.97,80.84,100.57


In [35]:
df_fatiga = df.groupby(["Ejercicio", "Fecha", "Peso (kg)"]).agg({"VMP (m/s)" : ["min", "max"]})
idx = df.groupby(["Ejercicio", "Fecha"])[["Peso (kg)"]].idxmax()
idx = idx.droplevel(2)  # Eliminar el nivel de índice adicional
df_peso_maximo = df_fatiga.loc[idx].reset_index()
df_peso_maximo = df_peso_maximo.drop(columns=["Peso (kg)"])


IndexError: Too many levels: Index has only 2 levels, not 3

In [34]:
df_fatiga = df.groupby(["Ejercicio", "Fecha", "Peso (kg)"]).agg({"VMP (m/s)" : ["min", "max"]})
idx = df.groupby(["Ejercicio", "Fecha"])[["Peso (kg)"]].idxmax()
df_peso_maximo = df_fatiga.loc[idx].reset_index()
df_peso_maximo = df_peso_maximo.drop(columns=["Peso (kg)"])


ValueError: Cannot index with multidimensional key

In [37]:
#Añado nuevas columnas con datos que me interesan
df_fatiga = df.groupby(["Ejercicio", "Fecha", "nº Serie", "Peso (kg)"]).agg({"VMP (m/s)" : ["min", "max"]})
df_fatiga["Fatiga"] = df_fatiga["VMP (m/s)"]["max"] - df_fatiga["VMP (m/s)"]["min"]
df_fatiga["Porcentaje de fatiga"] = df_fatiga["Fatiga"] / df_fatiga["VMP (m/s)"]["max"] * 100
#QUiero hacer bucle for para que me coja el valor máximo de peso de cada día y se lo paso a chatgpt y le pido que lo haga por pandas
df_fatiga


3                                                VMP (m/s)       Fatiga  \
                                                       min   max          
Ejercicio          Fecha      nº Serie Peso (kg)                          
SENTADILLA FRONTAL 2023-01-30 1        20.0           1.23  1.23   0.00   
                              2        50.0           0.79  0.90   0.11   
                              3        50.0           0.77  0.90   0.13   
                              4        50.0           0.79  0.91   0.12   
                   2023-02-07 1        20.0           0.61  1.27   0.66   
                              2        50.0           0.89  0.89   0.00   
                              3        60.0           0.78  0.78   0.00   
                              4        65.0           0.66  0.74   0.08   
                              5        70.0           0.58  0.68   0.10   
                              6        70.0           0.60  0.67   0.07   
                   2023-02-09 1        60.0           0.73  0.82   0.09   
                              2        60.0           0.69  0.83   0.14   
                              3        60.0           0.52  0.80   0.28   
                   2023-02-14 1        75.0           0.50  0.61   0.11   
                              2        75.0           0.50  0.60   0.10   
                              3        70.0           0.47  0.66   0.19   
                   2023-02-19 1        65.0           0.58  0.62   0.04   
                              2        60.0           0.62  0.75   0.13   
                              3        60.0           0.64  0.78   0.14   
                              4        60.0           0.61  0.76   0.15   
                   2023-02-21 1        60.0           0.57  0.71   0.14   
                              2        70.0           0.60  0.70   0.10   
                              3        75.0           0.46  0.59   0.13   
                              4        75.0           0.50  0.59   0.09   
                   2023-02-23 1        65.0           0.57  0.70   0.13   
                              2        65.0           0.63  0.80   0.17   
                              3        65.0           0.74  0.88   0.14   
                   2023-02-27 1        70.0           0.53  0.63   0.10   
                              2        75.0           0.56  0.64   0.08   
                              3        80.0           0.47  0.53   0.06   
                              4        80.0           0.52  0.63   0.11   
                   2023-03-01 1        70.0           0.52  0.62   0.10   
                              2        65.0           0.53  0.63   0.10   
                              3        65.0           0.70  0.76   0.06   
                   2023-03-06 1        60.0           0.68  0.78   0.10   
                              2        85.0           0.41  0.51   0.10   
                              3        80.0           0.50  0.57   0.07   
                              4        85.0           0.45  0.59   0.14   

3                                                Porcentaje de fatiga  
                                                                       
Ejercicio          Fecha      nº Serie Peso (kg)                       
SENTADILLA FRONTAL 2023-01-30 1        20.0                  0.000000  
                              2        50.0                 12.222222  
                              3        50.0                 14.444444  
                              4        50.0                 13.186813  
                   2023-02-07 1        20.0                 51.968504  
                              2        50.0                  0.000000  
                              3        60.0                  0.000000  
                              4        65.0                 10.810811  
                              5        70.0                 14.705882  
                              6        70.0                 

In [14]:
#Convierto dataframe a excel
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()
